In [1]:
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.inverse_kinematics import PositionConstraint, MinimumDistanceConstraint
from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder
from pydrake.solvers.mathematicalprogram import MathematicalProgram
from pydrake.math import RigidTransform
from pydrake.solvers.snopt import SnoptSolver
from pydrake.symbolic import Variable
from pydrake.all import eq
import numpy as np

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, MultibodyPlant(time_step=0.01))
parser = Parser(plant)
arm = parser.AddModelFromFile("../model/resources/allegro_hand_description/urdf/allegro_arm.urdf")

In [3]:
root = plant.GetBodyByName("hand_root")
root_frame = root.body_frame()
plant.WeldFrames(plant.world_frame(),
                 root_frame,
                 RigidTransform(p=np.zeros(3)))
plant.Finalize()

# Create Plant context
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
plant.SetPositions(plant_context, np.zeros(plant.num_positions(),))

In [4]:
# Setup mathematical problem
prog = MathematicalProgram()

# Add variables
q = np.empty((5, plant.num_positions()), dtype = Variable)
dq = np.empty((4, plant.num_positions()), dtype = Variable)
for i in range(5):
    q[i] = prog.NewContinuousVariables(plant.num_positions(), 'q'+str(i))

for i in range(4):
    dq[i] = prog.NewContinuousVariables(plant.num_positions(), 'dq'+str(i))

# Add constraints
joint_lower_limits = plant.GetPositionLowerLimits()
joint_upper_limits = plant.GetPositionUpperLimits()
print(joint_lower_limits)
print(joint_upper_limits)

# Transition constraints
prog.AddConstraint(eq(q[0], np.zeros(plant.num_positions())))
prog.AddConstraint(eq(q[4], 0.2 *np.ones(plant.num_positions())))
for i in range(4):
    prog.AddConstraint(eq(q[i+1], q[i]+dq[i]))
    prog.AddBoundingBoxConstraint(joint_lower_limits * 0.1, joint_upper_limits * 0.1, dq[i])

for i in range(5):
    prog.AddBoundingBoxConstraint(joint_lower_limits, joint_upper_limits, q[i])

# Add End effector position constraint
source_body = plant.GetBodyByName("link_15_tip")
source_frame = source_body.body_frame()

position_constraint = PositionConstraint(plant, 
                                             plant.world_frame(), 
                                             np.array([0.1, 0.2, 0.1])-0.01,
                                             np.array([0.1, 0.2, 0.1])+0.01,
                                             source_frame,
                                             np.zeros(3),
                                             plant_context)
no_collision_constraint = MinimumDistanceConstraint(plant, 1e-4, plant_context)
for i in range(5):
    prog.AddConstraint(position_constraint, q[i])
    prog.AddConstraint(no_collision_constraint, q[i])


[-3.1415    -3.1415926 -2.        -2.        -1.47      -0.47
 -0.196     -0.174     -0.227     -0.47      -0.196     -0.174
 -0.227     -0.47      -0.196     -0.174     -0.227     -0.11
 -0.105     -0.189     -0.162    ]
[3.1415    3.1415926 2.        2.        1.47      0.47      1.61
 1.709     1.618     0.47      1.61      1.709     1.618     0.47
 1.61      1.709     1.618     1.396     1.163     1.644     1.719    ]


In [5]:
solver = SnoptSolver()
result = solver.Solve(prog, np.random.random(9 * plant.num_positions()))

In [7]:
sol = result.GetSolution()

(189,)

In [ ]:
no_collision_constraint